## Follw the steps in this note book to download and process rainfall data for historical typhoon events 


The steps in this scripst 

* Import Libraries
* Define functions 
* Import Methadata for typhoon events (Name, start data, end date, Land fall ) if we want to add new event edit the typhoon_methdadta.csv 
* Download and process rainfall for typhoon events in the Metadata
* Store rainfall information for each Manucipalities 
    -- At the end of the script you will have two csv files with data for 
        6hr Maximum rainfall, 
        24hr Maximum rainfall and 
        Total rainfall for each manuciplaity and each Typhhon Event 

 
### Observed Total rainfall Example for few Typhoon events 
* Nesat and Nalgae Soak  https://earthobservatory.nasa.gov/images/52366/typhoons-nesat-and-nalgae-soak-the-philippines
* Fengshen https://earthobservatory.nasa.gov/images/related/8868/rainfall-from-typhoon-fengshen

In [1]:
import pandas as pd
import numpy as np
import random
import os
import sys
import datetime as dt
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from pandas.core.dtypes.missing import isnull
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import openpyxl
import seaborn as sns
from numpy.lib.shape_base import _expand_dims_dispatcher
import datetime
import math
import ftplib
import gzip
import zipfile
from ftplib import FTP_TLS
import ssl
import numpy as np
import rasterio
from fiona.crs import from_epsg
from rasterio.transform import array_bounds
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats
import shutil


# Setting directory

wor_dir='/home/mforooshani/Typhoon-Impact-based-forecasting-model-training-5:7/IBF-Typhoon-model/'

cdir=wor_dir
sys.path.insert(0, wor_dir+'src')
#from settings import GPM_USERNAME 
os.chdir(wor_dir)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from bs4 import BeautifulSoup
import requests
user, password = 'akliludin@gmail.com', 'akliludin@gmail.com'


def listFD(url,user, password, ext=''):
    page = requests.get(url,auth=(user, password)).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]


baseurl = 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata'
y=2021
m=4
d=13
file=f'/{y}/0{m}/{d}/gis'
url=baseurl+file
ext = 'tif'
fillist=[file for file in listFD(url, user, password,ext)]

In [3]:
fillist[0].split('/')[-1]

'3B-DAY-GIS.MS.MRG.3IMERG.20210413-S000000-E235959.3060.V06B.tif'

In [4]:
import requests

baseurl = 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata'

user, password = 'akliludin@gmail.com', 'akliludin@gmail.com'

r = requests.get(baseurl + file, auth=(user, password))

In [5]:
import requests

baseurl = 'https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata'
y=2021
m=4
d=13
file=f'/{y}/0{m}/{d}/gis'

user, password = 'akliludin@gmail.com', 'akliludin@gmail.com'

r = requests.get(baseurl + file, auth=(user, password))
 
#wor_dir='C:/Typhoon_IBF/Typhoon-Impact-based-forecasting-model/IBF-Typhoon-model/'

filepath=wor_dir+'test2.tif'

open(filepath, 'wb').write(r.content)

40526

## Functions to download and process Rainfall data 

Module to process the data for a typhoon that is required for feature building and
running the machine-learning pipeline.
There are a number of things that this module does:
(i) The methods download_gpm and download_srtm download the data for rainfall and elevation, respectively.
(ii) These data are then used, along with the windspeed and track data to obtain average values per municipality.
These data are then outputted into a CSV file for each typhoon that it is run for. It needs to be run
ONLY ONCE for each typhoon.
REQUIRED INPUTS:
(i) Typhoon name.
(ii) Start/Landfall/End date for the typhoon.
(iii) Landfall time
(iiii) IMERG data type: early, late or final: the data respository also needs to be checked, lest files are moved away
                        and the data doesn't exist anymore.
OUTPUTS:
    (i) Downloaded GPM data
    (ii) CSV file with hhr rainfall in mm/hr


In [6]:
#%% Functions used
def date_range(start_date, end_date):
    return [
        str(start_date + dt.timedelta(days=x))
        for x in range((end_date - start_date).days + 1)
    ]


def unzip(zip_file, destination):
    os.makedirs(destination, exist_ok=True)

    with zipfile.ZipFile(zip_file) as zf:
        zf.extractall(destination)

    return

def listFD(url,user, password, ext=''):
    try:
        page = requests.get(url,auth=(user, password)).text
        soup = BeautifulSoup(page, 'html.parser')
    except requests.exceptions.RequestException as error:
         print("Error:\n", error)
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

def reproject_file(gdf, file_name, force_epsg):

    print(
        "Reprojecting %s to EPSG %i...\n" % (file_name, force_epsg), end="", flush=True
    )
    gdf = gdf.to_crs(epsg=force_epsg)

    return gdf


def reproject_raster(
    src_array, src_transform, src_epsg, dst_epsg, src_nodata=-32768, dst_nodata=-32768
):
    """ Method to re-project a data-frame in the digital elevation model (DEM) to EPSG format.
    :param src_array: the data in DEM format
    :type src_array: pandas data-frame
    :param src_transform:
    :type src_transform:
    :param src_epsg:
    :type src_epsg:
    :param dst_epsg:
    :type dst_epsg:
    :param src_nodata:
    :type src_nodata:
    :param dst_nodata:
    :type dst_nodata:
    :raises:
    :returns:
    """
    src_height, src_width = src_array.shape
    dst_affine, dst_width, dst_height = calculate_default_transform(
        from_epsg(src_epsg),
        from_epsg(dst_epsg),
        src_width,
        src_height,
        *array_bounds(src_height, src_width, src_transform)
    )

    dst_array = np.zeros((dst_width, dst_height))
    dst_array.fill(dst_nodata)

    reproject(
        src_array,
        dst_array,
        src_transform=src_transform,
        src_crs=from_epsg(src_epsg),
        dst_transform=dst_affine,
        dst_crs=from_epsg(dst_epsg),
        src_nodata=src_nodata,
        dst_nodata=dst_nodata,
        resampling=Resampling.nearest,
    )

    return dst_array, dst_affine

 


def download_gpm_http(start_date, end_date, download_path, type_imerg):

    """ Method that downloads gpm files.
    This method looks in the data repositories of NASA for rainfall data.
    :param start_date: A date object denoting the START date to search for rainfall data.
    :param end_date: A date object denoting the END date to search for rainfall data.
    :param download_path: A string denoting where the data should be downloaded to.
    :param type_imerg: Hart-coded st
    :returns: file_list, a  list of files?
    :raises: ftplib.allerrors
    """
    # Username and password for logging in
    # Can create own account on NASA site
    ppm_username = 'akliludin@gmail.com'#GPM_USERNAME 
    base_url = ""

    # Connection to the site, if pasting in chrome: https://arthurhouhttps.pps.eosdis.nasa.gov/
    # Directory to where the data is saved
    if type_imerg == "final":
        base_url = "https://arthurhouhttps.pps.eosdis.nasa.gov/pub/gpmdata"

    date_list = date_range(start_date, end_date)
    file_list = []

    for date in date_list:
        print(date)
        d, m, y = reversed(date.split("-"))
        day_path = os.path.join(download_path, y + m + d)

        # Make a folder for each day, to save GPM data

        os.makedirs(day_path, exist_ok=True)
        if type_imerg == "final":
            data_dir_final=f'/{y}/{m}/{d}/gis'
            url=base_url+data_dir_final
            #ext = 'tif'
            tiff_files=listFD(url, user=ppm_username, password=ppm_username,ext='tif')

            for tifffile in tiff_files:
                file_name=tifffile.split('/')[-1]
                
                file_path=os.path.join(day_path,file_name)
                file_list.append(file_path)
                try:
                    r = requests.get(tifffile, auth=(ppm_username, ppm_username))
                    open(file_path, 'wb').write(r.content)
                except requests.exceptions.RequestException as error:
                        print("Error:\n", error)

    return file_list 

      




def cumulative_rainfall(admin_geometry, start_date, end_date, download_path, type_imerg):

    """ Method to calcualte the cumulative amount of rainfall from the typhoon.
    :param admin_geometry:
    :type admin_geometry:
    :param start_date: A date object signifying the start date of the typhoon.
    :param end_date: A date object signifying the end date of the typhoon.
    :param download_path: A string denoting where the data should be downloaded to.
    :param type_imerg: A string denoting the data file type: "early" or "final"
    :returns: sum_rainfall: list of precipitation rates in mm/hr for each half hour time interval
    :raises:
    """
    not_found_files = []

    # File_list is a list of tif files for each date
   
    file_list = download_gpm_http(start_date, end_date, download_path, type_imerg)

    sum_rainfall = []
    file_list = sorted(file_list)

    if file_list:

        print("Reading GPM data...\n", end="", flush=True)

        transform = ""
        # creating dataframe to save daily rainfall per municipality
        df_rainfall = pd.DataFrame(admin_gdf["ADM3_PCODE"])
        i = 1

        for input_raster in file_list:

            print(input_raster)

            try:

                with rasterio.open(input_raster) as src:

                    array = src.read(1)
                    transform = src.transform
                    sum_rainfall = zonal_stats(
                        admin_geometry,
                        array,
                        stats="mean",
                        nodata=29999,
                        all_touched=True,
                        affine=transform,
                    )

                    sum_rainfall = [i["mean"] for i in sum_rainfall]

                    # / 10 give mm/h because the factor in the tif file is 10 --> mm / h
                    def convert(x):
                        return x / 10

                    sum_rainfall_converted = [convert(item) for item in sum_rainfall]

                    # column name --> obtain time frame from input_raster name
                    # Time frame is preceded by '-S'
                    identify_str = ".3IMERG."
                    str_index = input_raster.find(identify_str)
                    len_date = 16  # length of string to include as name
                    column_name = input_raster[
                        str_index
                        + len(identify_str) : str_index
                        + len(identify_str)
                        + len_date
                    ]

                    df_rainfall[column_name] = sum_rainfall_converted

            except:

                print("not found")
                not_found_files.append(input_raster)

    else:
        print(
            "No files were found/downloaded from the appropriate folder. Please investigate further.\n"
        )
        pass

    return df_rainfall, not_found_files


def process_tyhoon_data(typhoon_to_process, typhoon_name):
    """ Method to process the data for typhoons.
    :param typhoon_to_process: A dictionary instance containing all required information about the data for the typhoon.
    :param typhoon_name: The name of the typhoon (can be just passed through as the dictionary key.
    """

    typhoon = typhoon_to_process.get("typhoon")

    # Start/End date for precipitation data, get from the dictionary
    start_date = min(typhoon_to_process.get("dates"))
    end_date = max(typhoon_to_process.get("dates"))
    print("start_date is:", start_date, "end date of typhoon is:", end_date)

    # IMERG data type, either "early" (6hr), "late" (18hr) or "final" (4 months),
    # see https://pps.gsfc.nasa.gov/Documents/README.GIS.pdf
    imerg_type = typhoon_to_process.get("imerg_type")  # "early"
    print("imerg_type:", imerg_type)

    # Specify P Coded column in administrative boundaries file
    p_code = "ADM3_PCODE"

    # Specify output file names
    output_matrix_csv_name = typhoon_name + "_matrix.csv"

    # Output will be in this CRS, datasets are reprojected if necessary
    force_epsg = 32651  # UTM Zone 51N

    t0 = dt.datetime.now()

    # Specify the names to save the GPM data (folder) and the output file
    subfolder = typhoon + "/"
    gpm_path = os.path.join(gpm_folder_path, subfolder, "GPM")
    output_matrix_csv_file = os.path.join(output_path, output_matrix_csv_name)

    # Calculating cumulative rainfall
    if not imerg_type == "trmm":
        df_rainfall, not_found_files = cumulative_rainfall(
            admin_geometry_wgs84, start_date, end_date, gpm_path, imerg_type
        )

    # TOD move the rainfall data into the other CSV file.
    if output_matrix_csv_name:
        print(
            "Exporting output to %s...\n" % output_matrix_csv_name, end="", flush=True
        )
        # output_df = output_gdf.drop("geometry", axis=1)
        df_rainfall.to_csv(output_matrix_csv_file, index=False)

    t_total = dt.datetime.now()
    print("Completed in %fs\n" % (t_total - t0).total_seconds(), end="", flush=True)

    return not_found_files

## Download GPM rainfall Data

In [7]:
#%%#########################
### FILL IN INPUT HERE ###
##########################

# Setting the number of days prior to the landfall data for which to collect data
days_to_landfall = 2

# Setting path to save the GPM data
gpm_file_name = "Training-data-new/data/rainfall_data/output_hhr/gpm"
gpm_folder_path = os.path.join(wor_dir, gpm_file_name)

# Setting path to save the obtained DataFrames
output_file_name = "Training-data-new/data/rainfall_data/output_hhr_processed"
output_path = os.path.join(wor_dir, output_file_name)

# Default = FALSE
# IMPORTANT: setting to TRUE means that all downloaded GPM files will be deleted and re-downloaded
delete_folders = False

# Setting directory for administrative boundaries shape file
admin_file_name = (
    "Training-data-new/data/phl_administrative_boundaries/phl_admbnda_adm3.shp"
)
admin_file = os.path.join(wor_dir, admin_file_name)

# Setting directory for typhoon metadata csv file
typhoon_metadata_filename = os.path.join(
    cdir, "Training-data-new/data/metadata_typhoons.csv"
)
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")
# To make sure the dates can be converted to date type
for i in range(len(typhoon_metadata)):
    typhoon_metadata["startdate"][i] = typhoon_metadata["startdate"][i].replace(
        "/", "-"
    )
    typhoon_metadata["enddate"][i] = typhoon_metadata["enddate"][i].replace("/", "-")
    typhoon_metadata["landfalldate"][i] = typhoon_metadata["landfalldate"][i].replace(
        "/", "-"
    )

## Define events for which you want to download rainfall data

In [8]:
# Typhoons for which to download rainfall data
typhoons = list(typhoon_metadata.typhoon.values)

# if there is already rainfall data in the project folder  output_hhr_processed
typhoons_with_rainfall_data = [items.split('_')[0] for items in os.listdir(wor_dir+'/Training-data-new/data/rainfall_data/output_hhr_processed/')]
#display(typhoons_with_rainfall_data)
typhoons_without_rainfall_data = [items for items in typhoons if items not in typhoons_with_rainfall_data]

In [9]:
typhoon_metadata_ = typhoon_metadata.set_index("typhoon").to_dict()
typhoons_dict = dict()


# Loading admin-files only once instead of every time
print("Importing adminfile...\n", end="", flush=True)
admin_gdf = gpd.GeoDataFrame()
try:
    admin_gdf = gpd.GeoDataFrame.from_file(admin_file)
except IOError as ioe:
    print("Could not load file properly", ioe, end="", flush=True)
# Check if CRS is defined and default to WGS 84 if not
if not admin_gdf.crs:
    admin_gdf.crs = from_epsg(4326)
# Keeping an unprojected copy of admin area geometry in WGS84 to speed up raster calculations
# .geometry --> gives the geometry column of the dataframe
if admin_gdf.crs != "EPSG:4326":
    admin_geometry_wgs84 = reproject_file(admin_gdf.geometry, admin_file_name, 4326)
else:
    admin_geometry_wgs84 = admin_gdf.geometry
    # Output will be in this CRS, datasets are reprojected if necessary
    force_epsg = 32651  # UTM Zone 51N
# Check CRS of each layer and reproject if necessary
if admin_gdf.crs != force_epsg:
    admin_gdf = reproject_file(admin_gdf, admin_file_name, force_epsg)

###################################################################
### START OF PROCESSING
###################################################################


for typhoon in typhoons_without_rainfall_data[1:]:
    case = typhoon
    typhoons_dict[case] = {
        "typhoon": typhoon,
        "dates": [
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date()- dt.timedelta(days=2),
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date()+ dt.timedelta(days=2),
            dt.datetime.strptime(
                typhoon_metadata_["landfalldate"][typhoon], "%d-%m-%Y"
            ).date(),
        ],
        "imerg_type": typhoon_metadata_["imerg_type"][typhoon],
    }

gpm_file_name = "Training-data-new/data/rainfall_data/output_hhr/gpm/"
gpm_folder_path = os.path.join(wor_dir, gpm_file_name)
output_path="Training-data-new/data/rainfall_data/output_hhr_processed"

for keys in typhoons_dict.keys():
    typhoon_to_process=typhoons_dict[keys]
    typhoon = typhoon_to_process.get("typhoon")

    # Start/End date for precipitation data, get from the dictionary
    start_date = min(typhoon_to_process.get("dates"))
    end_date = max(typhoon_to_process.get("dates"))
    print("start_date is:", start_date, "end date of typhoon is:", end_date)

    # IMERG data type, either "early" (6hr), "late" (18hr) or "final" (4 months),
    # see https://pps.gsfc.nasa.gov/Documents/README.GIS.pdf
    imerg_type = typhoon_to_process.get("imerg_type")  # "early"
    print("imerg_type:", imerg_type)

    # Specify P Coded column in administrative boundaries file
    p_code = "ADM3_PCODE"

    # Specify output file names
    output_matrix_csv_name = keys + "_matrix.csv"

    # Output will be in this CRS, datasets are reprojected if necessary
    force_epsg = 32651  # UTM Zone 51N

    t0 = dt.datetime.now()

    # Specify the names to save the GPM data (folder) and the output file
    subfolder = typhoon #+ "/"
    gpm_path = os.path.join(gpm_folder_path, subfolder, "GPM")
    output_path = os.path.join(wor_dir, output_path)
    output_matrix_csv_file = os.path.join(output_path, output_matrix_csv_name)

    # Calculating cumulative rainfall
    if not imerg_type == "trmm":
        df_rainfall, not_found_files = cumulative_rainfall(admin_geometry_wgs84, start_date, end_date, gpm_path, imerg_type)

    # TOD move the rainfall data into the other CSV file.
    if output_matrix_csv_name:
        print(
            "Exporting output to %s...\n" % output_matrix_csv_name, end="", flush=True
        )
        # output_df = output_gdf.drop("geometry", axis=1)
        df_rainfall.to_csv(output_matrix_csv_file, index=False)
    

Importing adminfile...
Reprojecting Training-data-new/data/phl_administrative_boundaries/phl_admbnda_adm3.shp to EPSG 32651...


### Process downloaded  rainfall data 

In [10]:
#%% Loading information sheet

typhoon_metadata_filename = os.path.join(
    wor_dir, "Training-data-new/data/metadata_typhoons.csv"
)
typhoon_metadata = pd.read_csv(typhoon_metadata_filename, delimiter=",")
display(typhoon_metadata)

# To make sure the dates can be converted to date type
for i in range(len(typhoon_metadata)):
    typhoon_metadata["startdate"][i] = typhoon_metadata["startdate"][i].replace(
        "/", "-"
    )
    typhoon_metadata["enddate"][i] = typhoon_metadata["enddate"][i].replace("/", "-")
    typhoon_metadata["landfalldate"][i] = typhoon_metadata["landfalldate"][i].replace(
        "/", "-"
    )

typhoon_metadata["landfall_date_time"] = (
    typhoon_metadata["landfalldate"] + "-" + typhoon_metadata["landfall_time"]
)

typhoons = list(typhoon_metadata["typhoon"].values) #typhoon_metadata["typhoon"] 
"""
Obtain 6, 24h maximum in mm/h and Total rainfall in mm
"""
#%% Processing the data into an excel sheet
time_frame = 48  # in half hours
time_frame_6 = 12  # in half hours
mov_window = 12  # in half hours
before_landfall_h = 72  # how many hours before landfall to include
after_landfall_h=72  # how many hours before landfall to include


num_intervals = math.floor((2 * 144 - time_frame) / mov_window) + 1 #144 to include 72hr before and after 
num_intervals_6 = math.floor((2 * 144 - time_frame_6) / mov_window) + 1 #144 to include 72hr before and after 


df_rainfall_final = pd.DataFrame(columns=["typhoon", "Mun_Code", "rainfall_Total"])

for typhoon in typhoons:

    # Getting typhoon info
    df_info = typhoon_metadata[typhoon_metadata["typhoon"] == typhoon]
    landfall = df_info["landfall_date_time"].values[0]
    landfall = datetime.datetime.strptime(landfall, "%d-%m-%Y-%H:%M:%S")

    # End date is landfall date
    # Start date is 72 hours before landfall date
    #end_date = landfall
    end_date = landfall + datetime.timedelta(hours=after_landfall_h) #landfall
    #start_date = end_date - datetime.timedelta(hours=before_landfall_h)
    start_date = landfall - datetime.timedelta(hours=before_landfall_h)

    # Loading the data
    file_name = (
        "Training-data-new/data/rainfall_data/output_hhr_processed/" + typhoon + "_matrix.csv"
    )
    path = os.path.join(cdir, file_name)
    df_rainfall = pd.read_csv(path)

    # Convert column names to date format
    for col in df_rainfall.columns[1:]:
        date_format = datetime.datetime.strptime(col, "%Y%m%d-S%H%M%S")
        df_rainfall = df_rainfall.rename(columns={col: date_format})

    df_mean_rainfall = pd.DataFrame({"Mun_Code": df_rainfall["ADM3_PCODE"]})
    available_dates_t = [date for date in df_rainfall.columns[1:] if (date >= start_date) & (date < end_date)]
    
    
    #########################################3
    
    df_mean_rainfall_6 = pd.DataFrame({"Mun_Code": df_rainfall["ADM3_PCODE"]})

    for i in range(num_intervals_6):

        start = start_date + datetime.timedelta(minutes=i * mov_window * 30)
        end = (
            start_date
            + datetime.timedelta(minutes=i * mov_window * 30)
            + datetime.timedelta(minutes=time_frame_6 * 30)
        )

        available_dates = [
            date for date in df_rainfall.columns[1:] if (date >= start) & (date < end)
        ]

        # To check if there is data available for all needed dates
        #if len(available_dates) != time_frame:
            #print("There are less files available than would be needed: please inspect")
            #print(typhoon, start, end, len(available_dates))

        df_mean_rainfall_6[i] = df_rainfall[available_dates].mean(axis="columns")
        
    #####################################

    df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
    
    
    df_mean_rainfall_24 = pd.DataFrame({"Mun_Code": df_rainfall["ADM3_PCODE"]})

    for i in range(num_intervals):

        start = start_date + datetime.timedelta(minutes=i * mov_window * 30)
        end = (
            start_date
            + datetime.timedelta(minutes=i * mov_window * 30)
            + datetime.timedelta(minutes=time_frame * 30)
        )

        available_dates = [
            date for date in df_rainfall.columns[1:] if (date >= start) & (date < end)
        ]

        # To check if there is data available for all needed dates
        #if len(available_dates) != time_frame:
            #print("There are less files available than would be needed: please inspect")
            #print(typhoon, start, end, len(available_dates))

        df_mean_rainfall_24[i] = df_rainfall[available_dates].mean(axis="columns")

    df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
    
    df_mean_rainfall["rainfall_Total"]= 0.5*df_rainfall[available_dates_t].sum(axis="columns")

    df_rainfall_single = df_mean_rainfall[["Mun_Code","rainfall_max_6h", "rainfall_max_24h","rainfall_Total"]]
    
    df_rainfall_single["typhoon"] = typhoon
    df_rainfall_final = df_rainfall_final.append(df_rainfall_single)


file_path = "Training-data-new/data/rainfall_data/rainfall_data.csv"  # in mm/h
df_rainfall_final.to_csv(file_path, index=False)


,typhoon,startdate,enddate,landfalldate,landfall_time,imerg_type
0,durian2006,24/11/2006,09/12/2006,30/11/2006,06:00:00,final
1,fengshen2008,17/06/2008,25/06/2008,20/06/2008,06:00:00,final
2,aere2011,05/05/2011,15/05/2011,07/05/2011,21:00:00,final
3,atsani2020,29/10/2020,07/11/2020,06/11/2020,00:00:00,final
4,bopha2012,25/11/2012,09/12/2012,03/12/2012,21:00:00,final
5,danas2019,14/07/2019,23/07/2019,17/07/2019,00:00:00,final
6,fung-wong2014,17/09/2014,25/09/2014,19/09/2014,03:00:00,final
7,goni2015,13/08/2015,30/08/2015,22/08/2015,00:00:00,final
8,goni2020,28/10/2020,06/11/2020,31/10/2020,21:00:00,final
9,hagupit2014,30/11/2014,12/12/2014,06/12/2014,18:00:00,final


/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat

/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat

/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat

/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat

/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat

/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_6h"] = df_mean_rainfall_6.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:122: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_mean_rainfall["rainfall_max_24h"] = df_mean_rainfall_24.max(axis="columns")
/tmp/ipykernel_628233/2288190635.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_rainfall_final = df_rainfall_final.append(df_rainfall_single)
/tmp/ipykernel_628233/2288190635.py:97: FutureWarning: Dropping of nuisance columns in Dat